In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = [8,6]
import os
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools
from tqdm import tqdm
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

In [ ]:
# Load data
train_df = pd.read_csv('../input/train_lbeltwx/train.csv')
test_df = pd.read_csv('../input/test_scvgim0/test.csv')
train_df.head()

In [ ]:
label_counts = train_df.label.value_counts()
plt.figure(figsize = (12,6))
sns.barplot(label_counts.index, label_counts.values, alpha = 0.9)
plt.xticks(rotation = 'vertical')
plt.xlabel('Image Labels', fontsize =12)
plt.ylabel('Counts', fontsize = 12)
plt.show()

In [ ]:
TRAIN_PATH = '../input/train_lbeltwx/train/'
TEST_PATH = '../input/test_scvgim0/test/'

In [ ]:
def read_image(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (28,28)) 
    return img

In [ ]:
train_data = []
test_data = []
train_labels = train_df['label'].values

for img in tqdm(train_df['id'].values):
    train_data.append(read_image(TRAIN_PATH + '{}.png'.format(img)))
    
for img in tqdm(test_df['id'].values):
    test_data.append(read_image(TEST_PATH + '{}.png'.format(img)))

In [ ]:
x_train = np.array(train_data, np.float32) / 255.
x_test = np.array(test_data, np.float32) / 255.

In [ ]:
test = train_df.label

In [ ]:
# One hot encoding dependent varible
test = to_categorical(test,num_classes=10)

In [ ]:
test[0]

In [ ]:

# check some images# check  
def show_images(ix):
    image_train = read_image(TRAIN_PATH + str(train_df.id[ix]) + '.png')
    image_test = read_image(TEST_PATH + str(test_df.id[ix]) + '.png')
    
    pair = np.concatenate((image_train, image_test), axis=1)
    
    plt.figure(figsize = (6,6))
    plt.imshow(pair)
    
# first 4 images in train and test set
for idx in range(4):
    show_images(idx)

In [ ]:
# Splitting data into training set and validation set
X_train, X_test, y_train, y_test = train_test_split(x_train,test,test_size = 0.3,random_state = 100)

In [ ]:
classifier = Sequential()

# adding 2 convolution layer and 1 maxpooling layer
classifier.add(Conv2D(filters = 32, kernel_size = (3,3),padding = 'Same', activation ='relu', input_shape = (28,28,3)))
classifier.add(Conv2D(filters = 32, kernel_size = (3,3),padding = 'Same', activation ='relu'))
classifier.add(MaxPool2D(pool_size=(2,2)))
classifier.add(Dropout(0.25))

# adding 2 convolution layer and 1 maxpooling layer
classifier.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
classifier.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
classifier.add(MaxPool2D(pool_size=(2,2)))
classifier.add(Dropout(0.25))

classifier.add(Flatten())
classifier.add(Dense(256, activation = "relu"))
classifier.add(Dropout(0.5))
classifier.add(Dense(10, activation = "softmax"))

In [ ]:
#Compile model
classifier.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
get_lr = ReduceLROnPlateau(monitor='val_acc', 
                            patience=3, 
                            verbose=1, 
                            factor=0.5, 
                            min_lr=0.00001)

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=10,
    zoom_range = 0.1,
    width_shift_range = 0.1,
    height_shift_range = 0.1
    )
datagen.fit(X_train)

In [ ]:
epochs = 40
batch_size = 100
classifier.fit_generator(datagen.flow(X_train,y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = (X_test,y_test),
                              verbose = 2,
                              callbacks=[get_lr]
                                )

In [ ]:
y_pred = classifier.predict(x_test)

In [ ]:
y_pred = np.argmax(y_pred,axis=1)

In [ ]:
output = pd.concat([test_df.id,pd.Series(y_pred,name ="label")],axis = 1)

In [ ]:
output.head()

In [ ]:
output.to_csv("2nd_submission.csv",index=False)